# Create samples from the test data

The goal of this notebook is to make samples out of the test data provided by the organizers.

As the test file is big (+6Gb, 3.5M objects), we will split the test data in smaller bunches that can be processed independently

In [2]:
import os, sys
import pandas as pd

import numpy as np
from util import *


# load the data. Obviously this is waaay to big to be put in memory, so we read it chunck by chunck

datapath = "data/test_set.csv"
metadatapath = "data/test_set_metadata.csv"

# df_chunks = pd.read_csv(datapath, chunksize=300)
# dfm = pd.read_csv(metadatapath)

# objs_ids = np.array(list(set(dfm["object_id"])))
# print("differents objects: ", len(objs_ids))
# print("id_min: ", min(float(v) for v in objs_ids))
# print("id_max: ", max(float(v) for v in objs_ids))


We have approx 3.5M objects. We'll put 10k objects per pickle, so ~350 pickles. Of course, the dumbfucks who created the challenge were not keen enough to label incrementally the ids...So we'll have to create a mapping function of our own



In [3]:
# initialize empty pickles

nop = 10000 # Number of Objects per Pickle
ids_chunks = [objs_ids[int(i*nop): int((i*nop)+10000)] for i in np.arange(len(objs_ids)/10000 + 1)]

#print len(ids_chunks)
#print len(list(set(ids_chunks[-1])))


def map_to_pkl(id):
	"""Return the matching pickle index for the given object id"""
	i, = np.where(objs_ids == id)[0]
	#print(i)
	pi = int(i/nop)
	return pi

#print(map_to_pkl(max(float(v) for v in objs_ids)))


In [4]:
import time
from tqdm import tqdm
import dask.dataframe as dd

import sys
# df = dd.read_csv(datapath)

# for chunk in pd.read_csv(datapath, chunksize=chunksize):
#     for i in tqdm(np.arange(chunksize)):
#         sample = chunk.iloc[i]
#         id = sample["object_id"]
#         pickle_id = map_to_pkl(id)
#     sys.exit()

# start = time.time
# for sample in df.itertuples():
# 	#id = sample["object_id"]
# 	#pi = map_to_pkl(id)
# 	pass
# stop = time.time()
# print(stop-start)

In [78]:
df = dd.read_csv("data/chunks/test_set_chunk0.csv")
dfm = dd.read_csv("data/chunks/test_set_chunk0_metadata.csv")

# pdfm = pd.read_csv("data/chunks/test_set_chunk0_metadata.csv")
# objs_ids = np.array(list(set(pdfm["object_id"])))
# print("differents objects: ", len(objs_ids))
# print("id_min: ", min(float(v) for v in objs_ids))
# print("id_max: ", max(float(v) for v in objs_ids))
# sys.exit()

ids_df = df["object_id"]
ids_dfm = dfm["object_id"]
unique_ids = list(set(ids_dfm.compute().values))
bands = [0, 1, 2, 3, 4, 5]

#print(df_samples.head(3))


In [112]:
#sample = dfm_sample.compute().to_dict()

#bmask = (df_samples["passband"] == 0)
#ss = df_samples[bmask]["mjd"]

import time
# start = time.time()
# df_samples.compute()
# 
# stop=time.time()
# print(stop-start)

# df_samples = df_samples.repartition(npartitions=1)
# dfm_sample = dfm_sample.repartition(npartitions=1)
# import time
from tqdm import tqdm as tqdm
series_list = []
for u_id in unique_ids[:2]:
	#start = time.time()	
	mask_df = (ids_df == u_id)
	df_samples = df[mask_df]

	mask_dfm = (ids_dfm == u_id)
	dfm_sample = dfm[mask_dfm]

	dfm_sample = dfm_sample.compute().iloc[0].to_dict()	
	#df_samples = df_samples.repartition(npartitions=1)
	df_samples = df_samples.compute()
	
	for b in bands:
		bmask = (df_samples["passband"] == b)
		dfm_sample["mjds_%s" % b] = df_samples[bmask]["mjd"].values
		dfm_sample["fluxes_%s" % b] = df_samples[bmask]["flux"].values
		dfm_sample["fluxerrs_%s" % b] = df_samples[bmask]["flux_err"].values
		dfm_sample["detected_%s" % b] = df_samples[bmask]["detected"].values
	
	#stop = time.time()
	#print(stop-start)	
	series_list.append(pd.Series(dfm_sample))
	
	
samples = pd.concat(series_list, axis=1).T
# convert what needs to be integer as integer
# ooh look Im pandas Im so fucking smart I save everything in float lolilol
samples[["object_id"]] = samples[["object_id"]].astype(int)	
	
	
print(samples)
# pickle the results for later use
writepickle(samples, "test_samples_test.pkl")
	
	
	

  ddf     decl                                         detected_0  \
0   1 -4.63048  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1   1 -63.4483  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_1  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_2  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_3  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_4  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_5  distmod  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..

In [71]:
for b in bands:
	bmask = (df_samples["passband"] == b)
	dfm_sample["mjds_%s" % b] = df_samples[bmask]["mjd"].values
	dfm_sample["fluxes_%s" % b] = df_samples[bmask]["flux"].values
	dfm_sample["fluxerrs_%s" % b] = df_samples[bmask]["flux_err"].values
	dfm_sample["detected_%s" % b] = df_samples[bmask]["detected"].values
	
	#dfm_sample.assign(df_samples[bmask]["mjd"])
	
	#sample["mjds_%s" % b] = df_samples[bmask]["mjd"].compute().values
	#sample["fluxes_%s" % b] 
	#mjds_band = dd.from_array(np.random.permutation(len(df)))
	#dfm_sample.insert(loc=0, column="mjds_%s" % b, values=df_samples[bmask]["mjds_%s" % b])	
print(dfm_sample)


  ddf     decl                                         detected_0  \
0   1 -4.63048  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1   1 -63.4483  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_1  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_2  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_3  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_4  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          detected_5  distmod  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..